<a href="https://colab.research.google.com/github/SlowikJakub/Homework4/blob/main/Problem2PartC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import collections
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [212]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [213]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [214]:
import torch.nn as nn
img, _ = cifar10[0]
pool=nn.MaxPool2d(2)
output=pool(img.unsqueeze(0))
img.unsqueeze(0).shape , output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

In [215]:
model= nn.Sequential(
    nn.Conv2d(3,16, kernel_size=3, padding=1),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Conv2d(3,16, kernel_size=3, padding=1),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Linear(8*8*8,32),
    nn.Tanh(),
    nn.Linear(32,4)
)

In [216]:
import torch.nn as nn
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(3,32, kernel_size=3, padding=1)
    self.act1=nn.Tanh()
    self.pool1=nn.MaxPool2d(2)
    self.conv2=nn.Conv2d(32,16, kernel_size=3, padding=1)
    self.act2=nn.Tanh()
    self.pool2=nn.MaxPool2d(2)
    self.conv3=nn.Conv2d(16,16, kernel_size=3, padding=1)
    self.act3=nn.Tanh()
    self.pool3=nn.MaxPool2d(2)
    self.fc1=nn.Linear(4*4*16,32)
    self.act4=nn.Tanh()
    self.fc2=nn.Linear(32,10)
  

  def forward(self,x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = self.pool3(self.act3(self.conv3(out)))
    out = out.view(-1, 4*4*16)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [217]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [218]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [219]:
import torch
import torch.nn as nn
import torch.optim as optim
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle =True)

model= Net().to(device=device)
optimizer = optim.SGD(model.parameters(),lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs=200,
    optimizer =optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader=train_loader
)

2021-04-23 00:55:44.420030 Epoch 1, Training loss 2.1074333828123635
2021-04-23 00:57:14.129464 Epoch 10, Training loss 1.2060491252890633
2021-04-23 00:58:51.889770 Epoch 20, Training loss 0.9448688880104543
2021-04-23 01:00:29.535914 Epoch 30, Training loss 0.8285440394030813
2021-04-23 01:02:07.370131 Epoch 40, Training loss 0.7598859531342831
2021-04-23 01:03:45.391792 Epoch 50, Training loss 0.7133285114756021
2021-04-23 01:05:22.848308 Epoch 60, Training loss 0.675482205036656
2021-04-23 01:06:59.343447 Epoch 70, Training loss 0.6468949716947877
2021-04-23 01:08:36.907175 Epoch 80, Training loss 0.6191193445792893
2021-04-23 01:10:14.723400 Epoch 90, Training loss 0.5976987599259447
2021-04-23 01:11:56.409471 Epoch 100, Training loss 0.5787103752727094
2021-04-23 01:13:35.897401 Epoch 110, Training loss 0.5602724673726674
2021-04-23 01:15:16.450826 Epoch 120, Training loss 0.5434890427171727
2021-04-23 01:16:57.686305 Epoch 130, Training loss 0.5273586926824602
2021-04-23 01:18:3

In [220]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=128,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=128,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

validate(model, train_loader, val_loader)


Accuracy train: 0.84
Accuracy val: 0.70


{'train': 0.83674, 'val': 0.6953}